In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [10]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [11]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [12]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130310

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,436617.3138
2018-02-28,456697.3772
2018-03-31,455876.8510
2018-04-30,368221.5081
2018-05-31,367161.2285
2018-06-30,368520.4841
2018-07-31,371361.1466
2018-08-31,681898.1097
2018-09-30,448512.3333


In [13]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    436617.3138
 2018-02-28    456697.3772
 2018-03-31    455876.8510
 2018-04-30    368221.5081
 2018-05-31    367161.2285
 2018-06-30    368520.4841
 2018-07-31    371361.1466
 2018-08-31    681898.1097
 2018-09-30    448512.3333
 2018-10-31    392051.7669
 2018-11-30    357668.7940
 2018-12-31    352197.9767
 2019-01-31    460636.2601
 2019-02-28    337108.5759
 2019-03-31    451311.0321
 2019-04-30    403901.9883
 2019-05-31    452586.0687
 2019-06-30    394032.0926
 2019-07-31    352706.3504
 2019-08-31    762261.7763
 2019-09-30    427957.7814
 2019-10-31    456351.5588
 2019-11-30    417377.4842
 2019-12-31    374690.8525
 2020-01-31    386177.0943
 2020-02-29    346026.6917
 2020-03-31    422649.5754
 2020-04-30    373785.8608
 2020-05-31    350237.8962
 2020-06-30    343827.7414
 2020-07-31    431939.1973
 2020-08-31    453846.2548
 2020-09-30    443241.5585
 2020-10-31    478496.8628
 2020-11-30    420604.2817
 2020-12-31    397758.8277

# ARIMA

In [14]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [15]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -6.191595
p-value : 0.000000
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [16]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1072.818, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1074.966, Time=0.03 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1077.293, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1078.062, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1077.076, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1117.808, Time=0.15 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1074.826, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1076.938, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1079.185, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1080.190, Time=0.08 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1076.855, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1078.907, Time=0.09 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1081.260, Time=0.13 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1082.837, Time=0.24 sec
 ARIMA(3,0,0)(0

ARIMA(order=(0, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [17]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 0)   Log Likelihood                -534.409
Method:                           css   S.D. of innovations          81233.301
Date:                Sat, 11 Sep 2021   AIC                           1072.818
Time:                        04:34:49   BIC                           1076.293
Sample:                             0   HQIC                          1074.091
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.285e+05   1.25e+04     34.184      0.000    4.04e+05    4.53e+05


C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [18]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([428481.04673095, 428481.04673095]), array([81233.30104616, 81233.30104616]), array([[269266.70233518, 587695.39112672],
       [269266.70233518, 587695.39112672]]))
